#Практика 2023

###Выполнила студентка группы БВТ2103 Железнова Светлана Ивановна


####Цель работы:
- создать программу, которая проверяет, которая по чек-листу проверяет, всё ли сказал работник колл-центра




####Задачи:
- изучить библиотеки, предназначенные для обработки естественного языка(nlp)
- проверить, все ли пункты чек-листа упомянул работник
- составить отчёты по конкретному разговору, по работе отдельного сотрудника и всего отдела за день

####Чек-лист:

1) Приветстие (В нашей компании сотрудники могут приветствовать клиетнов только словами "Здравствуйте или "Добрый день". Другие фразы неприемлемы)

2) Представление (Сотрудник должен представиться, назвать своё имя)

3) Упоминание ключевого слова (Сотруднику необходимо предложить клиенту приобрести автомобиль)

Для проверки первого пункта чек-листа я проверяю, есть ли в исходном тексте слова "Здравствуйте или "Добрый день".

Для проверки второго пункта используется распознавание именованных сущностей (DeepPavlov). Если в исходном тексте есть имя ('PER'), сотрудник представился.

Для проерки третьего пункта используются библиотеки Spacy и Gensim. Spacy используется для добавления части речи к слову, а библиотека Gensim необходима для оценки схожести слов.

####Отчёты

В отчёт по конкретному разговору включается:
- Информация о том, упомяул ли работник конкретны пункт чек-листа
- Качество работы сотрудника

В отчёт по работе сотрудника за день включается:
- Информация о том, сколько ошибок сотрудник сделал по каждому пункту чек-листа
- Гистограмма, содержащаю информацию о числе ошибок сотрудника
- Качество работы сотрудника

В отчёт по работе отдела (нескольких сотрудников) за день включается:
- Гистограмма, содержащаю информацию о качестве работы каждого сотрудника


#### Директория

Для корректной работы программы директория с файлами должна выглядеть следующим образом:


> дата (например 07.07.2023)
>> сотрудник_1

>>> запись_разговора_1.mp3

>>> запись_разговора_2.mp3

>>> ...

>> сотрудник_2

>>> запись_разговора_3.mp3

>>> ...

>> ...

####Отчёты

В результате выполнения программы будут созданы отчёты, расположение которых представлено ниже:

> дата (например 07.07.2023)
>> сотрудник_1

>>> отчёты_по_разговорам

>>>> **отчёт_по_разговору_1.docx**

>>>> **отчёт_по_разговору_2.docx**

>>> запись_разговора_1.mp3

>>> запись_разговора_2.mp3

>>>**отчёт_по_работе_сотрудника.docx**

>> сотрудник_2

>>> отчёты_по_разговорам

>>>> **отчёт_по_разговору_3.docx**

>>> запись_разговора_3.mp3

>>>**отчёт_по_работе_сотрудника.docx**

>> **отчёт по работе отдела.docx**

#####Сожержание отчётов

В отчёт по конкретному разговору включается:
- Информация о том, упомяул ли работник конкретны пункт чек-листа
- Качество работы сотрудника

В отчёт по работе сотрудника за день включается:
- Информация о том, сколько ошибок сотрудник сделал по каждому пункту чек-листа
- Гистограмма, содержащаю информацию о числе ошибок сотрудника
- Качество работы сотрудника

В отчёт по работе отдела (нескольких сотрудников) за день включается:
- Гистограмма, содержащаю информацию о качестве работы каждого сотрудника


####Ход работы

In [ ]:
!pip install -r requirements.txt

!python -m deeppavlov install ner_collection3_bert
!spacy-model install ru_core_news_sm

In [2]:
import whisper # для преобразования аудио в текст

# для распознавания именованных сущностей
from deeppavlov import configs
from deeppavlov import build_model

# для добавления части речи к слову
import spacy

# для оценки схожести слов
from gensim.models import Word2Vec
import gensim.downloader as api

import os # для работы с папками
import matplotlib.pyplot as plt # для гистограмм
from docx import Document # для создания отчётов в формате .docx

In [ ]:
whisper = whisper.load_model('medium')
ner = build_model('ner_collection3_bert', download=True, install=True)
nlp = spacy.load("ru_core_news_sm")
word2vec_ru = api.load("word2vec-ruscorpora-300")

models = {'whisper_model': whisper, 'ner_model': ner,
          'nlp': nlp, 'word2vec_model_ru': word2vec_ru}

In [3]:
# цвета для гистограмм
COLORS = ['#ffb399','#ffcc99','#ffe699',
          '#ffff99','#e6ff99','#ccff99',
          '#b3ff99','#99ff99','#99ffb3',
          '#99ffcc','#99ffe6','#99ffff',
          '#99e6ff','#99ccff','#99b3ff',
          '#9999ff','#b399ff','#cc99ff',
          '#e699ff','#ff99ff','#ff99e6',
          '#ff99cc','#ff99b3','#ff9999'
]

Функция check_audio необходима для проверки аудиозаписи (все ли пункты из чек-листа упомянуты)

In [5]:
def check_audio(file_name, models_to_use):
    whisper_model = models_to_use['whisper_model']
    ner_model = models_to_use['ner_model']
    nlp_model = models_to_use['nlp']
    word2vec_model_ru = models_to_use['word2vec_model_ru']

    # переводим аудио в текст
    res_whisper = whisper_model.transcribe(file_name)
    result_whisper = res_whisper['text'].replace('.', '')
    result_whisper = result_whisper.replace(', ', ' ')
    result_whisper = result_whisper.strip()

    # проверяем приветствие
    result_whisper_lower = result_whisper.lower()
    if ('здравствуйте' in result_whisper_lower or
        'добрый день' in result_whisper_lower):
        greeting = True
    else:
        greeting = False

    # проверяем, представился ли сотрудник
    ner_text = ner_model([result_whisper])[1][0]
    introduction = False
    for word in ner_text:
        if 'PER' in word:
            introduction = True
            break

    # схожие по смыслу слова
    # добавляем части речи
    doc = nlp(result_whisper)
    new_text = []
    for token in doc:
        new_text.append(token.text + '_' + token.pos_)
    # сравниваем по смыслу
    count = 0
    car_mentioned = False
    for word in new_text:
        if word in word2vec_model_ru:
            if word2vec_model_ru.similarity(word, 'машина_NOUN') >= 0.6:
                car_mentioned = True
                break

    audio_results = [greeting, introduction, car_mentioned]

    # создаём отчёт
    make_report_call(file_name, audio_results)

    return res_whisper['text'], audio_results

Функция make_report_call необходима для создания отчёта по отдельному разговору

In [6]:
def make_report_call(file, res):

        if '/' in file:
            # убираем дату и сотрудника из названия документа
            border = file.rfind('/')
            f_name = file[border + 1:-4] + '_call_report'
            file_path = file[:border] + '/' + 'call_reports/'
        else:
            f_name = file[:-4] + '_call_report'
            file_path = 'call_reports/'

        if not os.path.isdir(file_path):
              os.mkdir(file_path)

        file_name = file_path + f_name
        document = Document()
        document.add_heading('Отчёт по разговору ' +  f_name)
        count_mistakes = 0

        if res[0]:
              greet = '✅'
        else:
             greet = '❌'
             count_mistakes += 1
        document.add_paragraph(greet + ' Приветствие')

        if res[1]:
            intro = '✅'
        else:
            intro = '❌'
            count_mistakes += 1
        document.add_paragraph(intro + ' Представление')

        if res[2]:
            main_word = '✅'
        else:
            main_word = '❌'
            count_mistakes += 1
        document.add_paragraph(main_word + ' Упоминание ключевого слова(машина)')
        quality = (len(res) - count_mistakes)/len(res)
        document.add_paragraph('Качество работы струдника - '
                               + str(round(quality, 2)))

        document.save(file_name + '.docx')
        # print('ok')

Функция check_employee необходима для проверки работы сотрудника за день

In [7]:
def check_employee(file_dir, models_to_use):
    employee_res = []
    for f in os.scandir(file_dir):
        if f.is_file() and f.path.split('.')[-1].lower() == 'mp3':
            employee_res.append(check_audio(f.path, models_to_use)[1])

    greet_mistakes = 0
    intro_mistakes = 0
    main_word_mistakes = 0

    for call_res in employee_res:
        if not call_res[0]:
            greet_mistakes += 1
        if not call_res[1]:
            intro_mistakes += 1
        if not call_res[2]:
            main_word_mistakes += 1

    mistakes_arr = [greet_mistakes, intro_mistakes, main_word_mistakes]
    all_mistakes = greet_mistakes + intro_mistakes + main_word_mistakes
    all_items = len(employee_res)*len(employee_res[0])

    quality = ((all_items - all_mistakes)/all_items)

    make_report_employee(file_dir, mistakes_arr, quality)

    return mistakes_arr, quality

Функция make_report_employee необходима для создания отчёта по работе сотрудника за день

In [8]:
def make_report_employee(path, mistakes, work_quality):
        if '/' in path:
            # убираем дату из названия документа
            border = path.rfind('/')
            file_name = path + '/' + path[border:] + '_report'
            employee_name = path[11:]
        else:
            file_name = path + '/' + path + '_report'
            employee_name = path
        document = Document()
        document.add_heading('Отчёт по работе сотрудника ' +
                             employee_name + ' за день')
        document.add_paragraph()

        document.add_paragraph(str(mistakes[0]) +
                               ' - кол-во ошибок в приветствии')
        document.add_paragraph(str(mistakes[1]) +
                               ' - кол-во ошибок в представлении')
        document.add_paragraph(str(mistakes[2]) +
                               ' - кол-во ошибок в упоминании ключевого слова')

        document.add_paragraph()

        mistakes_names = ['Приветствие', 'Представление', 'Ключевое слово']
        # создаём гистограмму(горизонтальную)
        plt.bar(mistakes_names, mistakes, color = COLORS)
        # отображаем сетку
        plt.grid(linewidth = "0.5", linestyle = ":")
        # подписываем оси
        plt.xlabel('Ошибка')
        plt.ylabel('Число ошибок сотрудника, шт')
        # добавляем заголовок
        plt.title('Ошибки сотрудника')
        plt.savefig('saved_figure.png')
        plt.close()
        document.add_picture('./saved_figure.png')
        os.remove('saved_figure.png')

        document.add_paragraph('Качество работы струдника - '
                               + str(round(work_quality, 2)))

        document.save(file_name + '.docx')
        # print('ok 1')

Функция check_date необходима для проверки работы отдела (нескольких сотрудников) за день

In [9]:
def check_date(date_dir, models_to_use):
    qualities = []
    all_employees = []
    for dir in os.scandir(date_dir):
        if dir.is_dir() and ('.ipynb_checkpoints' not in str(dir)):
            qualities.append(check_employee(dir.path, models_to_use)[1])
            all_employees.append(str(dir)[11:-2])

    make_report_date(date_dir, qualities, all_employees)

Функция make_report_date необходима для проверки создания отчёта по работе отела за день

In [10]:
def make_report_date(date_dir, qualities, employees):
        # file_name = date_dir + '/' + 'reports/' + date_dir + '_report'
        file_name = date_dir + '/' + date_dir + '_report'
        document = Document()
        document.add_heading('Отчёт по работе отдела за ' +  date_dir)
        document.add_paragraph()

        # создаём гистограмму(горизонтальную)
        plt.bar(employees, qualities, color = COLORS)
        # отображаем сетку
        plt.grid(linewidth = "0.5", linestyle = ":")
        # подписываем оси
        plt.xlabel('Сотрудник')
        plt.ylabel('Качество работы сотрудника')

        # добавляем заголовок
        plt.title('Качество работы сотрудников')
        plt.savefig('saved_figure.png')
        plt.close()

        document.add_picture('./saved_figure.png')
        os.remove('saved_figure.png')

        document.add_paragraph()

        document.save(file_name + '.docx')
        # print('ok 2')

####Запуск программы

In [11]:
check_date('07.07.2023', models)

/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/